In [1]:
import torch
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PI = torch.tensor(np.pi, dtype=torch.float32, device=device)

def retrieve_name(var):
    import inspect
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

def log(*argv, show=False):
    for arg in argv:
        print(f"-"*75)
        print(f"{retrieve_name(arg)}")
        if show:
            print(f"content: ")
            print(arg)
        print(f"type: {type(arg)}")
        if isinstance(arg, np.ndarray) or isinstance(arg, torch.Tensor): 
            print(f"dtype: {arg.dtype}")
            print(f"shape: {arg.shape}")
        elif isinstance(arg, list) or isinstance(arg, str) or isinstance(arg, dict):
            print(f"len: {len(arg)}")

log(PI, show=True)

---------------------------------------------------------------------------
['PI']
content: 
tensor(3.1416, device='cuda:0')
type: <class 'torch.Tensor'>
dtype: torch.float32
shape: torch.Size([])


In [2]:
# NOTE The UPA mode only works with perfect square numbers; otherwise, 
#      it is rounded down to the nearest perfect square number.

def np_array_response(angle1: np.float32, angle2: np.float32, 
                      num: np.uint8, antenna_array: str) -> np.ndarray:
    """
    generate ULA and UPA steering vectors
    """
    assert num > 0
    
    array_response = np.zeros((num, 1), dtype=np.complex64)
    
    if antenna_array == 'UPA':
        num_sqrt = int(np.sqrt(num))
        assert num_sqrt * num_sqrt == int(num)

        for m in range(int(np.sqrt(num))):
            for n in range(int(np.sqrt(num))):
                array_response[m * (int(np.sqrt(num))) + n] = \
                    np.exp(1j*np.pi*(m*np.sin(angle1)*np.cos(angle2) + n*np.cos(angle2)))
    elif antenna_array == 'ULA':
        for n in range(num):
            array_response[n] = np.exp(1j*np.pi*(n*np.sin(angle1)))
   
    array_response = array_response / np.sqrt(num)
    return array_response

def torch_array_response(angle1: torch.FloatTensor, angle2: torch.FloatTensor, 
                         num: torch.ByteTensor, antenna_array: str) -> torch.Tensor:
    """
    generate ULA and UPA steering vectors
    """
    assert num > 0
    PI = torch.tensor(np.pi, dtype=torch.float32).to(device)
    
    array_response = torch.zeros((num, 1), dtype=torch.complex64).to(device)

    if antenna_array == 'UPA':
        num_sqrt = int(torch.sqrt(num))
        assert num_sqrt * num_sqrt == int(num)

        for m in range(num_sqrt):
            for n in range(num_sqrt):
                array_response[m * num_sqrt + n] \
                    = torch.exp(1j*PI*(m*torch.sin(angle1).to(device)*torch.cos(angle2).to(device) 
                                       + n*torch.cos(angle2).to(device))).to(device)
    elif antenna_array == 'ULA':
        for n in range(num):
            array_response[n] = torch.exp(1j*PI*(n*torch.sin(angle1).to(device))).to(device)
   
    array_response = array_response / torch.sqrt(num).to(device)
    return array_response

test_ula = np_array_response(0.56, 0, 3, 'ULA')
# log(test_ula)

test_ula = torch_array_response(
    torch.tensor(0.56, dtype=torch.float32), 
    torch.tensor(0, dtype=torch.float32), 
    torch.tensor(3, dtype=torch.uint8), 
    'ULA',
)
# log(test_ula)

test_upa = np_array_response(1.98, 2.67, 4, 'UPA')
# log(test_upa)

test_upa = torch_array_response(
    torch.tensor(1.989, dtype=torch.float32), 
    torch.tensor(2.67, dtype=torch.float32), 
    torch.tensor(4, dtype=torch.uint8), 
    'UPA',
)
# log(test_upa, show=True)


In [3]:
def np_ULA_response(angle: np.float32, num_antennas: np.uint8) -> np.ndarray:
    """
    Return the ULA steering vector

    Keyword arguments:
    angle:         the angles of arrival(AoA) or angle of departure (AoD) in radian
    num_antennas:  the number of Tx or Rx antennas
    """
    assert num_antennas > 0
    
    array_response = np.zeros((num_antennas, 1), dtype=np.complex64)
    
    for n in range(0, num_antennas):
        array_response[n] = np.exp(1j*np.pi*(n*np.sin(angle)))
    
    array_response = array_response / np.sqrt(num_antennas)
    return array_response

def np_UPA_response(azimuth: np.float32, elevation: np.float32, 
                    M_y: np.uint8, M_z: np.uint8) -> np.ndarray:
    """
    Return the UPA steering vector

    Keyword arguments:
    azimuth:    the azimuth AoA or AoD in radian
    elevation:  the elevation AoA or AoD in radian
    M_y:        the number horizontal antennas of Tx or Rx 
    M_z:        the number vertical antennas of Tx or Rx 
    """
    assert M_y > 0 and M_z > 0

    array_response = np.zeros((M_y * M_z, 1), dtype=np.complex64)
    
    for m in range(M_y):
        for n in range(M_z):
            array_response[m * M_z + n] = \
                np.exp(1j*np.pi*(m*np.sin(azimuth)*np.cos(elevation) + n*np.cos(elevation)))

    array_response = array_response / np.sqrt(M_y * M_z)
    return array_response

def np_USPA_response(azimuth: np.float32, elevation: np.float32, num_antennas: np.uint8) -> np.ndarray:
    """
    Return the Uniform Square Planar Array (USPA) steering vector

    Keyword arguments:
    azimuth:       the azimuth AoA or AoD in radian
    elevation:     the elevation AoA or AoD in radian
    num_antennas:  the total number of the Tx or Rx antennas
    """
    assert num_antennas > 0
    array_response = np.zeros((num_antennas, 1), dtype=np.complex64)
    
    num_sqrt = int(np.sqrt(num_antennas))
    assert num_sqrt * num_sqrt == int(num_antennas)

    for m in range(num_sqrt):
        for n in range(num_sqrt):
            array_response[m * num_sqrt + n] = \
                np.exp(1j*np.pi*(m*np.sin(azimuth)*np.cos(elevation) + n*np.cos(elevation)))

    array_response = array_response / np.sqrt(num_antennas)
    return array_response

test_ula = np_array_response(0.83, 0, 3, 'ULA')
# log(test_ula)

test_ula = np_ULA_response(0.83, 3)
test_ula_tensor = torch.from_numpy(test_ula).to(device)
# log(test_ula, test_ula_tensor)

test_upa = np_array_response(2.76, 3.12, 4, 'UPA')
# log(test_upa)

test_upa = np_UPA_response(2.76, 3.12, 2, 2)
test_upa_tensor = torch.from_numpy(test_upa).to(device)
# log(test_upa, test_upa_tensor)

In [4]:
def torch_ULA_response(angle: torch.FloatTensor, num_antennas: torch.ByteTensor) -> torch.Tensor:
    """
    Return the ULA steering vector

    Keyword arguments:
    angle:         the angles of arrival(AoA) or angle of departure (AoD) in radian
    num_antennas:  the number of Tx or Rx antennas
    """
    assert num_antennas > 0
    PI = torch.tensor(np.pi, dtype=torch.float32).to(device)
    
    array_response = torch.zeros((num_antennas, 1), dtype=torch.complex64).to(device)
    
    for n in range(0, num_antennas):
        array_response[n] = torch.exp(1j*PI*(n*torch.sin(angle).to(device))).to(device)
    
    array_response = array_response / torch.sqrt(num_antennas).to(device)
    return array_response

deg = torch.randint(low=0, high=360, size=(1,), dtype=torch.int16).to(device)
rad = torch.deg2rad(deg).to(device)

a1 = torch_array_response(
    rad, 
    torch.tensor(0, dtype=torch.float32).to(device), 
    torch.tensor(4, dtype=torch.uint8).to(device), 
    'ULA',
)
a2 = torch_ULA_response(
    rad, 
    torch.tensor(4, dtype=torch.uint8).to(device), 
)

# log(PI, deg, rad, a1, a2, show=True)

In [5]:
def torch_USPA_response(azimuth: torch.FloatTensor, elevation: torch.FloatTensor, 
                        num_antennas: torch.ByteTensor) -> torch.Tensor:
    """
    Return the Uniform Square Planar Array (USPA) steering vector

    Keyword arguments:
    azimuth:       the azimuth AoA or AoD in radian
    elevation:     the elevation AoA or AoD in radian
    num_antennas:  the total number of the Tx or Rx antennas
    """
    assert num_antennas > 0
    PI = torch.tensor(np.pi, dtype=torch.float32).to(device)
    
    array_response = torch.zeros((num_antennas, 1), dtype=torch.complex64, device=device)
    
    num_sqrt = int(torch.sqrt(num_antennas))
    assert num_sqrt * num_sqrt == int(num_antennas)

    for m in range(0, num_sqrt):
        for n in range(0, num_sqrt):
            array_response[m * num_sqrt + n] = \
                torch.exp(1j*PI*(m*torch.sin(azimuth).to(device)*torch.cos(elevation).to(device) 
                                 + n*torch.cos(elevation).to(device))).to(device)
            # print(f"{m * num_sqrt + n} = {m}*{num_sqrt} + {n}: {array_response[m * num_sqrt + n] / torch.sqrt(num_antennas)}")
    array_response = array_response / torch.sqrt(num_antennas).to(device)
    return array_response

deg1 = torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)
rad1 = torch.deg2rad(deg1).to(device)
deg2 = torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)
rad2 = torch.deg2rad(deg2).to(device)

a1 = torch_array_response(
    rad1, 
    rad2,
    torch.tensor(4, dtype=torch.uint8, device=device), 
    'UPA',
)
a2 = torch_USPA_response(
    rad1, 
    rad2,
    torch.tensor(4, dtype=torch.uint8, device=device), 
)
# log(deg1, rad1, deg2, rad2, a1, a2, show=True)

In [6]:
num = 16
num_sqrt = int(np.sqrt(num))
for m in range(0, num_sqrt):
    for n in range(0, num_sqrt):
        print(m * num_sqrt + n, end=" ")

print("\n")
M_y, M_z = 4, 5
for m in range(0, M_y):
    for n in range(0, M_z):
        print(m * M_z + n, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

In [7]:
# NOTE Be very careful when using a Tensor as an index; remember to first convert it to an Integer, 
#      otherwise we may encounter some undefined behavior

def torch_UPA_response(azimuth: torch.FloatTensor, elevation: torch.FloatTensor, 
                       M_y: torch.ByteTensor, M_z: torch.ByteTensor) -> torch.Tensor:
    """
    Return the UPA steering vector

    Keyword arguments:
    azimuth:    the azimuth AoA or AoD in radian
    elevation:  the elevation AoA or AoD in radian
    M_y:        the number horizontal antennas of Tx or Rx 
    M_z:        the number vertical antennas of Tx or Rx 
    """
    assert M_y > 0 and M_z > 0
    PI = torch.tensor(np.pi, dtype=torch.float32).to(device)
    
    array_response = torch.zeros((M_y * M_z, 1), dtype=torch.complex64, device=device)
    
    for m in range(0, M_y):
        for n in range(0, M_z):
            array_response[m * int(M_z) + n] = torch.exp(1j*PI*(m*torch.sin(azimuth).to(device)*torch.cos(elevation).to(device) 
                                                                + n*torch.cos(elevation).to(device))).to(device)
            # print(f"{m * M_z + n} = {m}*{M_z} + {n}: {array_response[m * int(M_z) + n] / torch.sqrt(M_y * M_z)}")
    array_response = array_response / torch.sqrt(M_y * M_z).to(device)
    return array_response


deg1 = torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)
rad1 = torch.deg2rad(deg1).to(device)
deg2 = torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)
rad2 = torch.deg2rad(deg2).to(device)

a1 = torch_array_response(
    rad1, 
    rad2,
    torch.tensor(4, dtype=torch.uint8, device=device), 
    'UPA',
)
a2 = torch_UPA_response(
    rad1, 
    rad2,
    torch.tensor(2, dtype=torch.uint8, device=device), 
    torch.tensor(2, dtype=torch.uint8, device=device), 
)
# log(deg1, rad1, deg2, rad2)
# log(a1, a2, show=True)

In [8]:
for i in range(1, 16 + 1):
    n = torch.tensor(i, dtype=torch.float32)
    n_sqrt = int(torch.sqrt(n))
    # if int(n / n_sqrt) * n_sqrt == i: print(f"{i}: {n_sqrt} x {int(n / n_sqrt)}")


In [9]:
for _ in range(3):
    rad = np.radians(np.random.randint(low=-180, high=180), dtype=np.float32)
    rad_tensor = torch.tensor(rad, dtype=torch.float32).to(device)
    # log(rad, rad_tensor)
    # print(rad_tensor.dtype)


In [10]:
def complex_Gaussian(mean, std, size=None):
    return np.random.normal(mean, std, size) + 1j*np.random.normal(mean, std, size)

size = 1
b = complex_Gaussian(0, 1)
b_tensor = torch.tensor(b, dtype=torch.complex64).to(device)
a = np.random.normal(0, 1, size=size) + 1j*np.random.normal(0, 1, size=size)
a_tensor = torch.from_numpy(a).to(device)
H_2_est = torch.from_numpy(complex_Gaussian(0, 1, (1, 1))).to(device)
channel_gain = torch.tensor(np.random.normal(0, 1) + 1j*np.random.normal(0, 1), dtype=torch.complex64)

# log(H_2_est, channel_gain)

In [11]:
def torch_CN(mean, std, size=None):
    return torch.normal(mean, std, size) + 1j*torch.normal(mean, std, size)

size = (1, )
a = torch_CN(0.0, 1.0, size=size)
b = torch_CN(0.0, 1.0, size=(2, 4))
log(a, b)

---------------------------------------------------------------------------
['a']
type: <class 'torch.Tensor'>
dtype: torch.complex64
shape: torch.Size([1])
---------------------------------------------------------------------------
['b']
type: <class 'torch.Tensor'>
dtype: torch.complex64
shape: torch.Size([2, 4])


In [12]:
def get_ULA_sample(num: int) -> torch.Tensor:
    return torch_ULA_response(
        torch.deg2rad(torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)).to(device),
        torch.tensor(num, dtype=torch.uint8, device=device), 
    )

def get_USPA_sample(num: int) -> torch.Tensor:
    return torch_USPA_response(
        torch.deg2rad(torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)).to(device),
        torch.deg2rad(torch.randint(low=0, high=360, size=(1,), dtype=torch.int16, device=device)).to(device),
        torch.tensor(num, dtype=torch.uint8, device=device), 
    )

a = get_ULA_sample(3)
b = get_USPA_sample(4)
log(a, b, show=True)

---------------------------------------------------------------------------
['a']
content: 
tensor([[0.5774+0.0000j],
        [0.5465-0.1862j],
        [0.4572-0.3525j]], device='cuda:0')
type: <class 'torch.Tensor'>
dtype: torch.complex64
shape: torch.Size([3, 1])
---------------------------------------------------------------------------
['b']
content: 
tensor([[ 0.5000+0.0000j],
        [-0.2706+0.4205j],
        [-0.0916+0.4915j],
        [-0.3638-0.3430j]], device='cuda:0')
type: <class 'torch.Tensor'>
dtype: torch.complex64
shape: torch.Size([4, 1])


In [13]:
Nk = 4
Nt = 8
M_y = M_z = 2
Ns = M_y * M_z

rad = np.radians(np.random.randint(low=-180, high=180, size=3), dtype=np.float32)

a_BS = np_ULA_response(angle=rad[0], num_antennas=Nt)
a_RIS = np_UPA_response(azimuth=rad[1], elevation=rad[2], M_y=M_y, M_z=M_z)
a_BS_H = a_BS.conjugate().T
a_RIS_H = a_RIS.conjugate().T
vec1 = a_RIS @ a_BS_H
print(f"a_BS.shape: {a_BS.shape}, a_RIS.shape: {a_RIS.shape}")
print(f"a_BS_H.shape: {a_BS_H.shape}, a_RIS_H.shape: {a_RIS_H.shape}")
# log(a_BS, a_RIS, a_BS_H, a_RIS_H)

H_1 = np.random.normal(0, 1, (Ns, Nt)) + 1j*np.random.normal(0, 1, (Ns, Nt)) 
# log(vec1, H_1)
print(f"vec1.shape: {vec1.shape}, H_1.shape: {H_1.shape}")


a_BS.shape: (8, 1), a_RIS.shape: (4, 1)
a_BS_H.shape: (1, 8), a_RIS_H.shape: (1, 4)
vec1.shape: (4, 8), H_1.shape: (4, 8)


In [14]:
# NOTE H_2 shape: (Nk, Ns)
def _get_USPA_sample(num):
    return np_USPA_response(
        np.radians(np.random.randint(low=-180, high=180), dtype=np.float32),
        np.radians(np.random.randint(low=-180, high=180), dtype=np.float32),
        num,
    )

def _get_ULA_sample(num):
    return np_ULA_response(
        np.radians(np.random.randint(low=-180, high=180), dtype=np.float32), 
        num
    )

a_RIS_H = np.zeros((1, Ns), dtype=np.complex64)
a_RIS_2 = np.zeros((Nk, Ns), dtype=np.complex64)
for i in range(0, Nk):
    a_RIS = _get_USPA_sample(Ns)
    a_RIS_H = a_RIS.conjugate().T
    a_RIS_2[i] = a_RIS_H

a_RIS_2 = torch.from_numpy(a_RIS_2).to(device)
a_RIS_H = torch.from_numpy(a_RIS_H).to(device)
# log(a_RIS_2, a_RIS_H)
print(f"a_RIS_H.shape: {a_RIS_H.shape}")
print(f"a_RIS_2.shape: {a_RIS_2.shape}")



a_RIS_H.shape: torch.Size([1, 4])
a_RIS_2.shape: torch.Size([4, 4])


In [15]:
# NOTE H_3 shape: (Nk, Nt)
a_BS_3 = np.zeros((Nk, Nt), dtype=np.complex64)
for i in range(Nk):
    a_BS_3[i] = _get_ULA_sample(Nt).conjugate().T
a_BS_3 = torch.from_numpy(a_BS_3).to(device)
# log(a_BS_3)
print(f"a_BS_3.shape: {a_BS_3.shape}")


a_BS_3.shape: torch.Size([4, 8])


In [16]:
from torch import distributions as dist

loc_mu = 0.43*PI
kappa_PE = torch.tensor(1.5, dtype=torch.float32, device=device)
dist.VonMises(loc_mu, kappa_PE).sample(sample_shape=(16,))


tensor([ 2.2930, -1.3396,  0.9611,  0.5071,  1.7226,  2.9626,  1.6058,  2.4264,
         0.3830,  0.4851,  1.6025, -2.5843, -2.0704,  1.5790,  1.8493, -2.5608],
       device='cuda:0')